In [21]:
import pandas as pd
import numpy as np
from config import config

In [3]:
peachtree_df = pd.ExcelFile(config['peach_tree_checks'])

peachtree_df = pd.read_excel(peachtree_df, dtype={"Check #": str,
                                                  "Cash Account": str})
peachtree_df = peachtree_df.iloc[:len(peachtree_df) - 2, :]
print(peachtree_df.dtypes)

Check #                 object
Date            datetime64[ns]
Payee                   object
Cash Account            object
Amount                 float64
dtype: object


In [19]:
void_mask = peachtree_df["Check #"].str.endswith("V")
check_no = peachtree_df.loc[void_mask, "Check #"].str.rstrip("V")


void_df = pd.DataFrame({'Check #': peachtree_df.loc[void_mask, "Check #"].str.rstrip("V"),
                        'Void': True,
                        'Void Date': peachtree_df.loc[void_mask, "Date"]
                        })
merged_df = pd.merge(peachtree_df, void_df, how='left')
merged_df.loc[merged_df['Void'].isna(), 'Void'] = False

# peachtree_df['void'] = False
# peachtree_df['void'][a] = True
# voided = peachtree_df[a]
# voided["Void Check"] = voided["Check #"]
# 

# voided["voided"] = voided["Check #"].str.rstrip("V")
#voided.loc[:, "Check #"] = check_no

In [20]:
merged_df

,Check #,Date,Payee,Cash Account,Amount,Void,Void Date
0,12580V,2023-10-09,The Shipley Group,11040,-15000.00,False,NaT
1,12583,2023-10-09,3460 Rockefeller CT LLC,11040,11000.00,False,NaT
2,12584,2023-10-09,AECOM Inc.,11040,82791.91,False,NaT
3,12585,2023-10-09,Balancelogic,11040,390.00,False,NaT
4,12586,2023-10-09,John V. Bean,11040,14.00,False,NaT
...,...,...,...,...,...,...,...
144,12696V,2023-12-15,American Express,11040,-31594.90,False,NaT
145,ON LINE PAY 12/22.23,2023-12-22,American Express,11040,16751.58,False,NaT
146,ON LINE PAY 12/23/23,2023-12-22,American Express,11040,14843.32,False,NaT
147,ON LINE PAY 12/23/24,2023-12-22,VOID,11040,NaN,False,NaT


In [21]:
merged_df.dtypes


Check #                 object
Date            datetime64[ns]
Payee                   object
Cash Account            object
Amount                 float64
Void                    object
Void Date       datetime64[ns]
dtype: object

---
## Sample Code

In [17]:
data1 = {'Code': [1, 2, 3],
         'Name': ['Company1', 'Company2', 'Company3'],
         'Value': [200, 300, 400], 
         'Extra Value': [17, 18, 19]
    }
df1 = pd.DataFrame(data1) #, columns= ['Code','Name','Value'])

data2 = {
         'Code': [2],
         'Name': ['Company2'],
         'Value': [1000],
    }

df2 = pd.DataFrame(data2) #, columns= ['Code','Name','Value'])

In [19]:
updated = df1.merge(df2, how='left', on='Code', suffixes=['', '_new'])
updated

,Code,Name,Value,Extra Value,Name_new,Value_new
0,1,Company1,200,17,NaN,NaN
1,2,Company2,300,18,Company2,1000.0
2,3,Company3,400,19,NaN,NaN


In [24]:
updated['Value'] = np.where(pd.notnull(updated['Value_new']), updated['Value_new'], updated['Value'])

In [25]:
updated

,Code,Name,Value,Extra Value,Name_new,Value_new
0,1,Company1,200.0,17,NaN,NaN
1,2,Company2,1000.0,18,Company2,1000.0
2,3,Company3,400.0,19,NaN,NaN


In [16]:
df1.index

Index([1, 2, 3], dtype='int64', name='Code')